# Childrens Act Disposals

In [59]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re

In [2]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [60]:
# Assigning databases and S3 bucket
db1 = "familyman_dev_v2"
fcsq_db = "fcsq"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
snapshot_date = "2021-08-19"

### Extracting disposals 

In [142]:
# The first select query extracts orders where the type of order is defined by the event model.
# The second select query extracts disposals where the type of order is determined by the value of the field model. Included in this are orders of no order (ORDNOM), orders refused (ORDREF), and 
#        orders withdrawn (G63). C43 contains child arrangement orders (contact/residence - the most common orders types in children act cases) 
# The cross join unnest function in the second select query splits the value (order types - where multiple values can be entered) into one row per value
d1 = f""" 
SELECT 
  event,
  case_number,
  event_model,
  receipt_date,
  entry_date,
  COALESCE(receipt_date, entry_date) AS disp_date,
  error,
  creating_court,
  CAST(NULL AS VARCHAR) AS field_model,
  CAST(NULL AS VARCHAR) AS all_ord_types,
  CAST(NULL AS VARCHAR) AS split_ord_type_value
FROM 
  {db1}.events 
WHERE 
  ((event_model IN ('C23','C25','C26','C27','C28','C29',
                        'C30','C31','C32A','C32B','C33','C34A','C34B','C35A','C35B','C37',
                        'C38A','C38B','C39','C40','C42','C43A','C44A','C44B','C45A','C45B',
                        'C53','C80','C81','C82') 
            AND error = 'N')
    OR (event_model = 'CPA' AND error = 'N' AND narrative Is Null))
  AND mojap_snapshot_date = date '{snapshot_date}'

UNION ALL

SELECT 
  e.event,
  e.case_number,
  e.event_model,
  e.receipt_date,
  e.entry_date,
  COALESCE(e.receipt_date, e.entry_date) AS disp_date,
  e.error,
  e.creating_court,
  f.field_model,
  f.value as all_ord_types,
  TRIM(ord_type) as split_ord_type_value
FROM 
  {db1}.events e
  INNER JOIN {db1}.event_fields f
    ON e.event = f.event
  CROSS JOIN UNNEST(SPLIT(f.value,',')) AS t(ord_type)  
WHERE 
  ((f.field_model IN ('C21_3', 'C24_6', 'C36_5', 'C43_5', 'ORDNOM_5', 'ORDREF_5'))
    AND error = 'N')
  AND e.mojap_snapshot_date = date '{snapshot_date}'
  AND f.mojap_snapshot_date = date '{snapshot_date}'
  
"""

pydb.create_temp_table (d1,"ca_disps")

In [143]:
pydb.read_sql_query ("select * from __temp__.ca_disps where receipt_date is null limit 10")

,event,case_number,event_model,receipt_date,entry_date,disp_date,error,creating_court,field_model,all_ord_types,split_ord_type_value
0,10002227478,FD01C00261,CPA,NaT,2003-09-11 15:19:46,2003-09-11 15:19:46,N,FD,<NA>,<NA>,<NA>
1,10002228023,FD01C00542,CPA,NaT,2003-09-11 17:10:04,2003-09-11 17:10:04,N,FD,<NA>,<NA>,<NA>
2,10002231327,FD00C10339,CPA,NaT,2003-09-15 13:08:22,2003-09-15 13:08:22,N,FD,<NA>,<NA>,<NA>
3,10002232940,FD00C10194,CPA,NaT,2003-09-16 09:56:18,2003-09-16 09:56:18,N,FD,<NA>,<NA>,<NA>
4,10002241606,FD94C02350,CPA,NaT,2003-09-19 11:08:45,2003-09-19 11:08:45,N,FD,<NA>,<NA>,<NA>
5,10002241611,FD94C02326,CPA,NaT,2003-09-19 11:09:09,2003-09-19 11:09:09,N,FD,<NA>,<NA>,<NA>
6,10002242222,FD94C01513,CPA,NaT,2003-09-19 12:37:54,2003-09-19 12:37:54,N,FD,<NA>,<NA>,<NA>
7,10002250859,FD98C00637,CPA,NaT,2003-09-24 10:59:11,2003-09-24 10:59:11,N,FD,<NA>,<NA>,<NA>
8,10002259917,FD96C00608,CPA,NaT,2003-09-26 15:03:02,2003-09-26 15:03:02,N,FD,<NA>,<NA>,<NA>
9,10002260922,FD97C00475,CPA,NaT,2003-09-27 09:25:45,2003-09-27 09:25:45,N,FD,<NA>,<NA>,<NA>


### Extracting children

In [144]:
d2 = f"""
  SELECT 
    a.*,
    f.value children,
    TRY_CAST(TRIM(child_role_id) as bigint) child_role_id
  FROM 
    __temp__.ca_disps a
    LEFT JOIN {db1}.event_fields f
      ON f.event = a.event
   CROSS JOIN UNNEST(SPLIT(f.value,',')) AS t(child_role_id)
  WHERE 
     f.field_model IN('C21_CH','C23_CH','C24_CH','C25_CH','C26_CH','C27_CH','C28_CH','C29_CH',
                      'C30_CH','C31_CH','C32A_CH','C32B_CH','C33_CH','C34A_CH',
                      'C34B_CH','C35A_CH','C35B_CH','C36_CH','C37_CH','C38A_CH',
                      'C38B_CH','C39_CH','C40_CH','C42_CH','C43_CH',
                      'C43A_CH','C44A_CH','C44B_CH','C45A_CH','C45B_CH','C53_CH',
                      'C80_CH','C81_CH','C82_CH','CPA_CH','ORDNOM_CH','ORDREF_CH')
    AND child_role_id <> ''
    AND f.mojap_snapshot_date = DATE'{snapshot_date}'
"""

pydb.create_temp_table (d2,"ca_disps_child_event")

In [29]:
pydb.read_sql_query ("select * from __temp__.ca_disps_child_event limit 20")

,event,case_number,event_model,receipt_date,entry_date,error,creating_court,field_model,all_ord_types,split_ord_type_value,children,child_role_id
0,20200073853,GH03P00262,C21,2003-10-31,2003-10-31 08:34:27,N,GH,C21_3,A,A,20200024130,20200024130
1,20301509889,GL19P00567,C21,2019-12-05,2019-12-10 12:08:51,N,GL,C21_3,A,A,10609813,10609813
2,20800185315,GG04P00573,C21,2005-02-14,2005-02-16 15:36:11,N,GG,C21_3,A,A,20800054471,20800054471
3,20800187702,GG05P00183,C21,2005-03-15,2005-03-16 11:57:30,N,GG,C21_3,A,A,20800060354,20800060354
4,20801061006,GG14P00637,C21,2015-12-02,2015-12-09 10:56:25,N,GG,C21_3,A,A,5470091,5470091
5,21100291998,GU06P00354,C21,2006-12-08,2006-12-12 09:00:35,N,GU,C21_3,A,A,21100074179,21100074179
6,21100576952,NZ12C00115,C21,2012-11-28,2012-12-20 17:29:20,N,GU,C21_3,A,A,2975518,2975518
7,21101426315,GU21C00067,C21,2021-05-21,2021-05-28 10:00:30,N,GU,C21_3,A,A,12021069,12021069
8,21500191539,HL08C10030,C21,2008-09-30,2008-10-01 10:18:07,N,HL,C21_3,A,A,21500060394,21500060394
9,21500221131,HL10P00026,C21,2010-03-17,2010-03-18 09:22:58,N,HL,C21_3,A,A,21500069784,21500069784


In [145]:
d3 = f"""
  SELECT
    a.*,
    r.role child_role_id,
    p.dob,
    p.gender,
    r.delete_flag
  FROM
    __temp__.ca_disps a
    JOIN {db1}.roles r 
      on a.case_number = r.case_number
    JOIN {db1}.parties p 
      on r.party = p.party 
  WHERE
    event not in (SELECT event FROM __temp__.ca_disps_child_event)
    AND role_model in ('CHLDC', 'CHLDZ')
    AND r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
"""

pydb.create_temp_table(d3, "ca_disps_child_case")

In [35]:
pydb.read_sql_query ("select count (*) count from __temp__.ca_disps_child_event")

,count
0,4084655


In [146]:
d4 = f"""
  SELECT 
    a.case_number,
    a.disp_date,
    EXTRACT(YEAR FROM a.disp_date) AS Year,
    EXTRACT(QUARTER FROM a.disp_date) AS Quarter,
    a.receipt_date,
    a.entry_date,
    a.event,
    a.field_model,
    a.split_ord_type_value,
    a.child_role_id,
    p.dob,
    p.gender,
    a.creating_court,
    r.delete_flag
  FROM
    __temp__.ca_disps_child_event a
    JOIN {db1}.roles r on a.child_role_id = r.role
    JOIN {db1}.parties p on r.party = p.party
  WHERE
    r.mojap_snapshot_date = DATE'{snapshot_date}'
    AND p.mojap_snapshot_date = DATE'{snapshot_date}'
  UNION ALL
  SELECT
    a.case_number,
    a.disp_date,
    EXTRACT(YEAR FROM a.disp_date) AS Year,
    EXTRACT(QUARTER FROM a.disp_date) AS Quarter,
    a.receipt_date,
    a.entry_date,
    a.event,
    a.field_model,
    a.split_ord_type_value,
    a.child_role_id,
    a.dob,
    a.gender,
     a.creating_court,
    a.delete_flag
  FROM
    __temp__.ca_disps_child_case a
"""

pydb.create_temp_table(d4, "ca_disps_all_children")

In [147]:
pydb.read_sql_query("select count(*) count from __temp__.ca_disps_all_children")

,count
0,15060353


### Extracting final orders

In [148]:
d5 = f"""
  SELECT 
    a.*,
    b.value AS final_order
  FROM 
    __temp__.ca_disps_all_children a
    LEFT JOIN {db1}.event_fields b
      on a.event = b.event
  WHERE 
      b.field_model IN ('C21_FO','C23_FO','C24_FO','C25_FO','C26_FO','C27_FO','C28_FO','C29_FO','C30_FO',
                      'C31_FO','C32A_FO','C32B_FO','C33_FO','C34A_FO','C34B_FO','C35A_FO','C35B_FO','C36_FO',
                      'C37_FO','C38A_FO','C38B_FO','C39_FO','C40_FO','C42_FO','C43_FO','C43A_FO','C44A_FO',
                      'C44B_FO','C45A_FO','C45B_FO','C53_FO','C80_FO','C81_FO','C82_FO','CPA_FO',
                      'ORDNOM_FO','ORDREF_FO')
              
    AND b.mojap_snapshot_date = DATE'{snapshot_date}'
"""

pydb.create_temp_table(d5, "ca_final_orders")

In [149]:
pydb.read_sql_query("select count(*) count from __temp__.ca_final_orders")

,count
0,13178431


In [150]:
pydb.read_sql_query("select * from __temp__.ca_disps_all_children where case_number not in (Select case_number from __temp__.ca_final_orders) limit 10")

,case_number,disp_date,year,quarter,receipt_date,entry_date,event,field_model,split_ord_type_value,child_role_id,dob,gender,creating_court,delete_flag
0,FD02P01749,2002-11-07,2002,4,2002-11-07,2002-11-07 14:03:52,10001660102,<NA>,<NA>,10000395167,2002-11-06,1,FD,N
1,FD02P01749,2003-04-25,2003,2,2003-04-25,2003-04-30 11:54:42,10001980533,<NA>,<NA>,10000395167,2002-11-06,1,FD,N
2,FD02P01749,2004-08-11,2004,3,2004-08-11,2004-08-11 16:58:22,10002914805,<NA>,<NA>,10000395167,2002-11-06,1,FD,N
3,FD02P01749,2003-12-31,2003,4,2003-12-31,2004-01-05 10:30:36,10002451081,<NA>,<NA>,10000395167,2002-11-06,1,FD,N
4,BD10C10062,2010-04-13,2010,2,2010-04-13,2010-04-13 16:22:37,14100633176,<NA>,<NA>,14100134347,2005-06-28,1,LS,N
5,MA98C08210,1999-03-08,1999,1,1999-03-08,1999-03-08 00:00:00,26200046968,<NA>,<NA>,26200077816,1996-09-26,2,MA,N
6,MA98C08200,1999-04-12,1999,2,1999-04-12,1999-04-12 00:00:00,26200046616,<NA>,<NA>,26200077732,1997-04-06,2,MA,N
7,MA98C08200,1999-04-12,1999,2,1999-04-12,1999-04-12 00:00:00,26200046612,<NA>,<NA>,26200077731,1989-10-06,1,MA,N
8,MA97C08047,1998-02-09,1998,1,1998-02-09,1998-02-09 00:00:00,26200046935,<NA>,<NA>,26200076031,1984-02-08,2,MA,N
9,MA98C08252,1999-04-26,1999,2,1999-04-26,1999-04-26 00:00:00,26200046971,<NA>,<NA>,26200077926,1998-03-17,1,MA,N


In [111]:
d5 = f"""
  SELECT 
    event,
    field_model,
    value,
    CASE WHEN field_model IN ('G50_AT', 'U22_AT')
          THEN split_value
          ELSE CAST (NULL AS VARCHAR)
        END AS order_type,
    CASE WHEN field_model NOT IN ('G50_AT', 'U22_AT')
          THEN value 
          ELSE CAST (NULL AS VARCHAR)
        END AS final_order
  FROM 
    {db1}.event_fields
    CROSS JOIN UNNEST(SPLIT(value,',')) AS t(split_value)
  WHERE 
      field_model IN ('G50_AT', 'U22_AT',
     
                      'C21_FO','C23_FO','C24_FO','C25_FO','C26_FO','C27_FO','C28_FO','C29_FO','C30_FO',
                      'C31_FO','C32A_FO','C32B_FO','C33_FO','C34A_FO','C34B_FO','C35A_FO','C35B_FO','C36_FO',
                      'C37_FO','C38A_FO','C38B_FO','C39_FO','C40_FO','C42_FO','C43_FO','C43A_FO','C44A_FO',
                      'C44B_FO','C45A_FO','C45B_FO','C53_FO','C80_FO','C81_FO','C82_FO','CPA_FO',
                      'ORDNOM_FO','ORDREF_FO','G63_F')
              
    AND mojap_snapshot_date = DATE'{snapshot_date}'
"""

pydb.create_temp_table(d5, "ca_final_orders")

In [104]:
# linking final order info and withdrawn app type from apps data
d6 = f"""
  SELECT 
    a.*,
    f.final_order,
    f.order_type AS order_type
  FROM 
    __temp__.ca_disps_all_children a
    LEFT JOIN __temp__.ca_final_orders f
      ON a.event = f.event
    LEFT JOIN {fcsq_db}.ca_apps_order_types w
      ON a.split_ord_type_value = CAST(w.event AS VARCHAR)
"""

pydb.create_temp_table(d6, "ca_disps_fo_wthd_app_type")

In [106]:
pydb.read_sql_query ("select count(*) count from __temp__.ca_disps_fo_wthd_app_type where field_model = 'G63_1' and order_type is null limit 10")

,count
0,12221


In [89]:
pydb.read_sql_query (f"""select * from {db1}.event_fields where field_model IN ('G50_AT','U22_AT') and event = 38801008221 """)

,value,field_model,event,bis_first_created_datetime,bis_last_modified_datetime,modified_by_process_name,inserted_by_process_name,mojap_file_land_timestamp,mojap_snapshot_date
0,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1612283626,2021-11-11
1,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1612283626,2021-12-08
2,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1612283626,2021-02-02
3,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1612283626,2021-03-18
4,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1612283626,2021-08-19
5,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1612283626,2021-09-14
6,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1612283626,2021-10-11
7,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1604076967,2020-10-30
8,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1604076967,2020-12-17
9,CNO,G50_AT,38801008221,NaT,NaT,<NA>,<NA>,1612283626,2021-07-08


### Withdrawn events 

In [132]:
d10 = f"""
  SELECT 
    a.event,
    a.value AS app_event_number,
    b.value AS children,
    child_split,
    c.value AS final_order
  FROM 
    {db1}.event_fields a
    LEFT JOIN {db1}.event_fields b
      ON a.event = b.event
    LEFT JOIN {db1}.event_fields c
      ON a.event = c.event
    CROSS JOIN UNNEST(SPLIT(b.value,',')) AS t(child_split)  
  WHERE
    a.field_model = 'G63_1' AND
    b.field_model = 'G63_CH' AND
    c.field_model = 'G63_FO'
    AND a.mojap_snapshot_date = date '{snapshot_date}'
    AND b.mojap_snapshot_date = date '{snapshot_date}'
    AND c.mojap_snapshot_date = date '{snapshot_date}'
      
"""

pydb.create_temp_table(d10, "ca_disps_wthdrw_events")

In [134]:
pydb.read_sql_query ("select * from __temp__.ca_disps_wthdrw_events limit 20")

,event,app_event_number,children,child_split,final_order
0,13000120253,13000113003,13000030464,13000030464,Y
1,14100171270,14100163860,14100039592,14100039592,Y
2,24302818532,24302653815,1458057,1458057,Y
3,24402065560,24401906190,7418012,7418012,Y
4,27800411406,20200061957,20200022753,20200022753,Y
5,28300318275,28300315733,28300052418,28300052418,Y
6,28500615166,28500609374,28500116719,28500116719,Y
7,16401773490,16401745814,10056983,10056983,Y
8,23901036920,23901016085,3647732,3647732,Y
9,28603037269,28602967692,9427889,9427889,Y


## Thoughts

In [ ]:
#### 
1. Extract Disposals
2. Withdrawn
3. Link withdrawn events to app types (can use apps as left join and filter on G63?)
4. Join disposals and withdrawn events
5. Extract children info (all and withdrawn)
6. link children to disps
7. Extract FO info (all and withdrawn)
8. link FO to disps

In [63]:
pydb.create_temp_table(
f""" 
SELECT 
  g.event,
  g.field_model as g_field_model,
  g.value,
  g2.value as g2_value,
  g2.field_model as g2_field_model
FROM 
  {db1}.event_fields g
  full outer join {db1}.event_fields g2
    on g.value = CAST (g2.event AS VARCHAR)
WHERE 
  ((g.field_model = 'G63_1' AND g2.field_model IN ('G50_AT','U22_AT')) OR (g.field_model = 'G63_1' AND g2.field_model IS NULL))
  AND g.mojap_snapshot_date = date '{snapshot_date}'
   AND g2.mojap_snapshot_date = date '{snapshot_date}'
  
""",

"ca_join_tests")
  